<a href="https://colab.research.google.com/github/fromakim/2021Election_Analysis/blob/main/data_getfollowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Dev Environment

In [1]:
import itertools
import json
import requests
from time import sleep
from urllib import parse
from datetime import datetime, timedelta

In [2]:
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/gdrive')
# LAMBDA: import boto3

Mounted at /content/gdrive


In [ ]:
# LABMDA: s3 = boto3.client('s3')

# 2. Key File Setting

In [5]:
f = open('./gdrive/MyDrive/Colab Notebooks/key.json')
# LAMBDA: f = open('./key.json')
keys = json.load(f)

In [6]:
api_key = keys['api_key']
api_secret_key = keys['api_secret_key']
bearer_token = keys['bearer_token']
app_id = keys['app_id']
index = int(keys['index'])

In [7]:
headers = {"Authorization" : f"Bearer {bearer_token}"}

# 3. Import Data

In [8]:
users = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/election_sample/user.csv')
# LAMBDA: users = pd.read_csv('./user.csv')

In [9]:
users

,name,verified,created_at,description,profile_image_url,username,id,url,protected,location,pinned_tweet_id
0,크즈,False,2020-03-09T05:58:08.000Z,대충 살아볼게,https://pbs.twimg.com/profile_images/126376587...,unkogej,1236893975488778241,NaN,False,NaN,NaN
1,눈을불알임,False,2020-12-29T17:35:56.000Z,그래도 해야지,https://pbs.twimg.com/profile_images/145519795...,himzo_,1343974013232562176,NaN,False,Republic of Korea,NaN
2,춰츄,False,2018-08-18T12:52:02.000Z,취향계,https://pbs.twimg.com/profile_images/119158039...,animal_heayo,1030799481199984640,NaN,False,NaN,1.113309e+18
3,에케,False,2013-06-09T04:53:28.000Z,트위터 3회차 | I:H:트리 | 0n년생 트위치 제휴 스트리머 에케입니다!\n국회...,https://pbs.twimg.com/profile_images/131676783...,sh4cker,1494643164,https://t.co/qx0JL74b7E,False,파란대나무숲,1.157582e+18
4,말랑,False,2018-01-27T20:31:28.000Z,NaN,https://pbs.twimg.com/profile_images/104685903...,renew0426,957350363669004288,NaN,False,NaN,9.826737e+17
...,...,...,...,...,...,...,...,...,...,...,...
12140,얼렁뚱땅,False,2020-08-07T09:03:43.000Z,🌕🌖🌗🌘🌑🌒🌓🌔🌕 개잡덕 / 허락 안받아도 됨 알아서 퍼가세요,https://pbs.twimg.com/profile_images/145543816...,ulttung,1291661243543990272,NaN,False,NaN,1.452117e+18
12141,박동건,False,2020-05-31T11:06:40.000Z,#민주당 대선후보 누가되던 절대지지# 정치를 외면하면 당신이 바보들에게 통지 당하고...,https://pbs.twimg.com/profile_images/139457839...,KalNicolas1,1267049788102422529,NaN,False,대한민국 울진군,NaN
12142,😈홍홍😈,False,2012-02-01T03:28:35.000Z,NaN,https://pbs.twimg.com/profile_images/144818627...,monoka21,480088821,NaN,False,대한민국,NaN
12143,우리는할 수 있습니다,False,2019-08-27T07:36:45.000Z,NaN,https://abs.twimg.com/sticky/default_profile_i...,alr8774k6kEdcrk,1166253104120991744,NaN,False,NaN,NaN


# 4. Collect Follow Network

In [ ]:
follows = pd.json_normalize(requests.get(f'https://api.twitter.com/2/users/{users["id"][index]}/following?max_results=1000', headers = headers).json()['data'])

In [ ]:
follows.columns = ['to', 'name', 'username']

In [ ]:
follows['from'] = users['id'][index]

In [ ]:
body = follows.to_json(orient = 'records')

'[{"to":"982126734","name":"\\u1d55\\u2240 \\u0320\\u1d55","username":"wouldujustnot","from":1236893975488778241},{"to":"977544986148274177","name":"MJ","username":"KIM_etal","from":1236893975488778241},{"to":"150633655","name":"Martin McKee","username":"martinmckee","from":1236893975488778241},{"to":"1181519339050156032","name":"June","username":"demipointe_139","from":1236893975488778241},{"to":"188244198","name":"jee","username":"limjees","from":1236893975488778241},{"to":"1209816676016873473","name":"\\uba74","username":"choimyeon","from":1236893975488778241},{"to":"275551267","name":"\\uae40\\uc5f0\\uacbd kim yeon koung","username":"kimyk10","from":1236893975488778241},{"to":"132762348","name":"JOY","username":"BF_cordwainer","from":1236893975488778241},{"to":"151764715","name":"\\uc0dd\\uac15","username":"inkivaariyksi","from":1236893975488778241},{"to":"43509988","name":"tablo","username":"blobyblo","from":1236893975488778241},{"to":"2972799367","name":"\\uc815\\ubc00\\uc544Jeon

In [ ]:
# LAMBDA: s3.put_object(Body = body, Bucket = 'electiondatafile', Key = f'follows/{users["id"][index]}.json')

# 5. Modify Key Settings

In [ ]:
keys['index'] = str(int(keys['index']) + 1)

In [ ]:
# LAMBDA: s3.put_object(Body = json.dumps(keys), Bucket = 'electiondatafile', Key = 'key.json')

Loop

In [10]:
follows = pd.DataFrame(columns = ['to_id', 'to_name', 'to_username', 'from_id', 'from_name', 'from_username'])

In [11]:
for index, user in users[234:].iterrows():
    try:
        print(f'user analysis on index [{index}]')

        res = requests.get(f'https://api.twitter.com/2/users/{user.id}/following?max_results=1000', headers = headers).json()

        data = res['data']
        count = res['meta']['result_count']
        
        df = pd.json_normalize(data)
        df = df[['id', 'name', 'username']]
        df.columns = ['to_id', 'to_name', 'to_username']
        
        df['from_id'] = user['id']
        df['from_name'] = user['name']
        df['from_username'] = user['username']

        follows = pd.concat([follows, df])

    except:
        print(f'execution stopped at the index : [{index}]')
        print(res)

        # if res['errors'][0]['title'] == 'Authorization Error':
        #     sleep(90)
        #     continue
        continue

    finally:
        sleep(90)

user analysis on index [234]
user analysis on index [235]
execution stopped at the index : [235]
{'errors': [{'resource_id': '1448998565976494080', 'parameter': 'id', 'resource_type': 'user', 'section': 'data', 'title': 'Authorization Error', 'value': '1448998565976494080', 'detail': 'Sorry, you are not authorized to see the user with id: [1448998565976494080].', 'type': 'https://api.twitter.com/2/problems/not-authorized-for-resource'}]}
user analysis on index [236]
user analysis on index [237]
user analysis on index [238]
user analysis on index [239]
user analysis on index [240]
user analysis on index [241]
user analysis on index [242]
user analysis on index [243]
user analysis on index [244]
user analysis on index [245]
user analysis on index [246]
user analysis on index [247]
user analysis on index [248]
user analysis on index [249]
user analysis on index [250]
user analysis on index [251]
user analysis on index [252]
user analysis on index [253]
execution stopped at the index : [25

KeyboardInterrupt: ignored

In [12]:
follows

,to_id,to_name,to_username,from_id,from_name,from_username
0,1294616936106475522,🪴비핀🌵,91Leex,27821557,심상정,sangjungsim
1,1367467910860394497,AH!,Ah92295606,27821557,심상정,sangjungsim
2,931842077259788289,구독만 할게요ㅜㅜ,gudog_u,27821557,심상정,sangjungsim
3,1187246818758819840,조각🍰,O_khee_,27821557,심상정,sangjungsim
4,1222116743826378752,말괄량이다람지,jaehoonjoon,27821557,심상정,sangjungsim
...,...,...,...,...,...,...
166,1312105238183071744,유덕창 ☃︎,udc00c,1332573919518236675,마구,magugyeni
167,1323416359514984451,주야,dayandnightzoo,1332573919518236675,마구,magugyeni
168,1264632529346195456,여모,_minyi_03,1332573919518236675,마구,magugyeni
169,1031006128144961536,VERIVERY_OFFICIAL,the_verivery,1332573919518236675,마구,magugyeni


In [13]:
users[380:386]

,name,verified,created_at,description,profile_image_url,username,id,url,protected,location,pinned_tweet_id
380,한산도,False,2021-07-15T03:51:54.000Z,노무현 문재인 대통령님을 모욕한 혜경궁 넌 누구냐?,https://pbs.twimg.com/profile_images/142850582...,LXPn7Pe5maPId77,1415519386383749124,NaN,False,NaN,NaN
381,카르텔v2.0,False,2017-04-18T12:26:09.000Z,구독계 플텍계 기타 내멋대로 블언블. 구독계라 의심되도 블언블 또는 차단\n\n사회...,https://pbs.twimg.com/profile_images/145652695...,_jbdjljm__,854310076546404352,NaN,False,NaN,1.454243e+18
382,토끼풀뜯어먹는소리🌊,False,2017-08-27T03:33:29.000Z,토끼는당근을싫어해🐰,https://pbs.twimg.com/profile_images/901662924...,letmeknowurs,901648838439575552,NaN,False,토끼의꿈이자라는호놀룰루,NaN
383,올리두,False,2021-03-28T13:03:39.000Z,갓생꿈나무,https://pbs.twimg.com/profile_images/137617441...,Olli_doo,1376157980635189252,NaN,False,NaN,1.381090e+18
384,마구,False,2020-11-28T06:36:05.000Z,최고의 짱돌 ♡,https://pbs.twimg.com/profile_images/141803005...,magugyeni,1332573919518236675,NaN,False,계프,1.444980e+18
385,b,False,2020-11-01T10:30:34.000Z,—̳͟͞͞♡ 한소희,https://pbs.twimg.com/profile_images/135731887...,bmiin_,1322848457380438017,NaN,False,NaN,1.451797e+18


In [14]:
follows.reset_index(drop = True).to_csv('followmap.csv')

In [ ]:
follows

,to_id,to_name,to_username,from_id,from_name,from_username
0,982126734,ᵕ≀ ̠ᵕ,wouldujustnot,1236893975488778241,크즈,unkogej
1,977544986148274177,MJ,KIM_etal,1236893975488778241,크즈,unkogej
2,150633655,Martin McKee,martinmckee,1236893975488778241,크즈,unkogej
3,1181519339050156032,June,demipointe_139,1236893975488778241,크즈,unkogej
4,188244198,jee,limjees,1236893975488778241,크즈,unkogej
...,...,...,...,...,...,...
104,1445600122427957253,혜견,G_211006,1445587101790007302,냠,ddinni_
105,1233190331761381377,🤍말랑공주❤,MeEl10_,1445587101790007302,냠,ddinni_
106,1232912988459290624,클튜출석해💢무해,MUHE_FF14,1445587101790007302,냠,ddinni_
107,724774289765896192,루토스,Rutos_0814,1445587101790007302,냠,ddinni_
